In [1]:
import os
from pathlib import Path
import splitfolders
from zipfile import ZipFile

In [2]:
DATA_PATH = Path("C:/Users/USER/Downloads/archive_1")

In [3]:
DATA_PATH


WindowsPath('C:/Users/USER/Downloads/archive_1')

In [4]:
Path.cwd(

)

WindowsPath('c:/Users/USER/Desktop/Farm-Insects-Classification/notebook')

In [5]:
os.chdir("../src/FarmInsectsClassifier/")

In [6]:
Path.cwd()

WindowsPath('c:/Users/USER/Desktop/Farm-Insects-Classification/src/FarmInsectsClassifier')

In [7]:
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    unzip_dir: Path

In [14]:
from FarmInsectsClassifier.constants import CONFIG_FILE_PATH, PARAM_FILE_PATH
from FarmInsectsClassifier.utils import create_directories, read_yaml, get_size

In [15]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 param_file_path = PARAM_FILE_PATH):
        
        
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAM_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            unzip_dir = Path(config.unzip_dir)

        )

        return data_ingestion_config

In [ ]:
class DataIngestion:
    def __init__(self, data_path: Path, config: DataIngestionConfig):
        self.data_path = data_path
        self.config = config

    def unzip(self) -> None:
        unzip_path = self.config.unzip_dir
        unzip_path.mkdir(exist_ok=True, parents=True)

        with ZipFile(self.data_path, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

    def split_data(self) ->  None:
        path = self.config.unzip_dir.resolve()
        splitfolders.ratio(path, seed=1, output="farm-insects-splitted", ratio=(0.6, 0.2, 0.2))